In [ ]:
#you have to admit the fact is that, you have to use rotation angle as the intersection bridge 
#since you cant find the quaternion of another point by the given quaternion of current point
#the logic is quaternion --> rotation angle + position info ---> quaternion
#or you can simple transfer everything into the form of rotation angle + position info

In [ ]:
#logistic
o1_position(c1, o1, depth1)
find_o2 (relpose_R, relpose_T, o1)
rotation_matrix_from_vectors(o1, o2, relpose_T)
cal_quaternion (relpose_T, o1, o2)

#or rotation_matrix_from_vectors(o1, o2, relpose_T) could be nested in cal_quaternion

In [ ]:
#relpose vectors for cameras 1,2,3,

scene_id= 0
main_loop_freq= 50.0
unity_render= yes 

fov= 90
width= 720
height= 480
num_camera= 3
camera_names= ["camera_1", "camera_2", "camera_3"]

camera_1:
  relpose_T: [0.0, 0.0, 0.3]
  relpose_R: [1.0, 0.0, 0.0, 0.0]

camera_2: 
  relpose_T: [-3.0, 0.0, 0.1]
  relpose_R: [0.924, 0.0, 0.0, 0.383] 

camera_3:
  relpose_T: [3.0, 0.0, 0.1]
  relpose_R: [0.924, 0.0, 0.0,-0.383]
    
# In my interpretation, the value of angle is from -1 to 1

In [ ]:
#inverse of a matrix and vector :numpy.linalg.inv()


In [1]:
#quaternion array in pyquaternion version
from pyquaternion import Quaternion

camera_1_r = Quaternion(axis=[1, 0, 0], angle=3.14159265) #angle range is from -pi to pi
camera_2_r = Quaternion(axis=[0, 1, 0], angle=3.14159265)
camera_3_r = Quaternion(axis=[0, 0, 1], angle=3.14159265)

#print(camera_1_r.angle)




#fixing the angle in this section

In [11]:
print(type(camera_1_r))

<class 'pyquaternion.quaternion.Quaternion'>


Given Info:  <br>
The position of cameras: relpose_T_1, relpose_T_2.... <br>
the quaternion or rotation matrix of camera 1 from the object in phase 1 to the same object in phase 2<br>
the depth of the object in phase 1 and the depth of the object in phase 2, hence we can calculate the positions of o1 and o2 <br>
the initial angle vector of camera1 to the object in phase 1, hence, we can calculate the vector c1o1<br>


In [ ]:

#defining the shape in this section
#define the relative postions function (shape such line, triangle...)
def triangular (relpose_T): 
    # first calculate the relative spatial position
    # second, add the noise to each position
    relpose_T_1 = [camera_1_relpose_T[0],  camera_1_relpose_T[1],  camera_1_relpose_T[2]]
    relpose_T_2 = [camera_1_relpose_T[0]-3.0, camera_1_relpose_T[1],camera_1_relpose_T[2] - 0.2]
    relpose_T_3 = [camera_1_relpose_T[0]+3.0, camera_1_relpose_T[1],camera_1_relpose_T[2] - 0.2]
    
    noise = np.random.normal(0,0.01,9) #generating 9 noise factors with standard deviation 0.01 at mean 0
    #output is 
    relpose_T_1 = [camera_1_relpose_T[0] + noise[0] ,  camera_1_relpose_T[1] + noise[1] ,  camera_1_relpose_T[2] + noise[2]]
    relpose_T_2 = [camera_1_relpose_T[0]-3.0 + noise[3] , camera_1_relpose_T[1] + noise[4] ,camera_1_relpose_T[2] - 0.2 + noise[5]]
    relpose_T_3 = [camera_1_relpose_T[0]+3.0 + noise[6] , camera_1_relpose_T[1] + noise[7] ,camera_1_relpose_T[2] - 0.2 + noise[8]]

def line (relpose_T): 
    # similarly, we define a line-shaped camera system for the multi cameras
    noise = np.random.normal(0,0.01,9)
    relpose_T_1 = [camera_1_relpose_T[0] + noise[0] ,  camera_1_relpose_T[1] + noise[1] ,  camera_1_relpose_T[2] + noise[2]]
    relpose_T_2 = [camera_1_relpose_T[0]+1.0 + noise[3] , camera_1_relpose_T[1]+1.0 + noise[4] ,camera_1_relpose_T[2] - 0.2679  + noise[5]]
    relpose_T_3 = [camera_1_relpose_T[0]+2.0 + noise[6] , camera_1_relpose_T[1]+2.0 + noise[7] ,camera_1_relpose_T[2] - 2.0  + noise[8]]



In [ ]:
def o1_position(c1, o1, depth1):
    c1o1 = [o1[0]-c1[0], o1[1]-c1[1], o1[2]-c1[2]]
    c1o1 = vec1 / np.linalg.norm(vec1)).reshape(3) #calculating the unit vector
    o1 = [c1[0] + c1o1[0] * depth1, c1[1] + c1o1[1] * depth1, c1[2] + c1o1[2] * depth1] #object position in phase 1
#using depth1 and initial angle of c1 to calculate position of o1

In [ ]:
def find_o2 (relpose_R, relpose_T, o1):
        #o1 = [a1, b1, c1]
        
#relpose_R is quaternion, relpose_T is rotation matrix
    x = relpose_R[0]
    y = relpose_R[1]
    z = relpose_R[2]
    w = relpose_R[3]
    
#converting from quaternion to rotation matrix
    R = [
        [1- 2*relpose_R[1]*relpose_R[1] - 2*relpose_R[2]*relpose_R[2], 2*relpose_R[0]*relpose_R[2] - 2*relpose_R[2]*relpose_R[3], 2*relpose_R[0]*relpose_R[2] + 2*relpose_R[1]*relpose_R[3]],
        [2*relpose_R[0]*relpose_R[1] + 2*relpose_R[2]*relpose_R[3], 1-2*relpose_R[0]*relpose_R[0]-2*relpose_R[2]*relpose_R[2], 2*relpose_R[1]*relpose_R[2] - 2*relpose_R[0]*relpose_R[3]],
        [2*relpose_R[0]*relpose_R[2] - 2*relpose_R[1]*relpose_R[3], 2*relpose_R[1]*relpose_R[2] + 2*relpose_R[0]*relpose_R[3], 1- 2*relpose_R[0]*relpose_R[0] -2*relpose_R[1]*relpose_R[1]]
    ]

#vector p = c1O1 (p is a vector from point c1 to point o1)
    p = [o1[0] - relpose_T[0], o1[1] - relpose_T[1], o1[2] - relpose_T[2]]
    
#p' = R*p
    second_vector = [
        R[0][0]*p[0] + R[0][1]*p[1] + R[0][2]*p[2],
        R[1][0]*p[0] + R[1][1]*p[1] + R[1][2]*p[2],
        R[2][0]*p[0] + R[2][1]*p[1] + R[2][2]*p[2]
        ]
    
#finding the unit vector of vector p'(second_vector)
    second_vector = second_vector / np.linalg.norm(second_vector)
    o2 = [relpose_T[0] + second_vector[0] * depth2, 
          relpose_T[1] + second_vector[1] * depth2,
          relpose_T[1] + second_vector[1] * depth2
         ] #object position in phase 2 (after the camera is rotated)
    
#now you have known o1 and o2 
#you can use them to calculate rotation matrix for c2 and c3
#or calculate the quaternion of c2 and c3


In [ ]:
import numpy as np

#this funtion below is to use o1 o2, and any camera location to calculate the rotation matrix needed
def rotation_matrix_from_vectors(o1, o2, relpose_T):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    vec1 = [o1[0] - relpose_T[0], o1[1] - relpose_T[1], o1[2] - relpose_T[2]]
    vec2 = [o2[0] - relpose_T[0], o2[1] - relpose_T[1], o2[2] - relpose_T[2]]
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    #ok, -v[2] in python means -v3 in math equations
    #https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [ ]:
from scipy.spatial.transform import Rotation as R
#calculating the quaternion for corresponding camera 2, camera 3, ..., simply substitude the camera position relpose_T of each camera
def cal_quaternion (relpose_T, o1, o2):
    rotation_matrix_from_vectors(o1, o2, relpose_T)
    rotation_matrix.as_quat() #converting the rotation matrix to quaternion
    relpose_R = rotation_matrix.as_quat()
    return relpose_R 
    #return the quaternion